In [1]:
%load_ext nb_black

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

import pymc3 as pm
import pandas as pd
import numpy as np
import seaborn as sns
import arviz as az
import theano
import theano.tensor as tt
import matplotlib.pyplot as plt
import statsmodels.stats.api as sms
import altair as alt
import json
import gzip, pickle


WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


<IPython.core.display.Javascript object>

In [2]:
model_data = pd.read_table("./data/update_simple.tab", sep="\t", index_col=0)
model_data["count_scaled"] = model_data["count"] / 10000
model_data

,count,day,week,count_scaled
0,158016,Nov 25,1,15.8016
1,218733,Dec 2,2,21.8733
2,268026,Dec 9,3,26.8026
3,311895,Dec 16,4,31.1895
4,358098,Dec 23,5,35.8098
...,...,...,...,...
138,2406672,Jul 18,139,240.6672
139,2413273,Jul 25,140,241.3273
140,2419440,Aug 01,141,241.9440
141,2425048,Aug 08,142,242.5048


<IPython.core.display.Javascript object>

In [3]:
len_observed = len(model_data)

sets = ["CotA", "AoA", "WC", "MM", "DT"]
week_of_release = [0, 27, 50, 85, 126]

with pm.Model() as model_6:
    # priors
    sigma = pm.Exponential("sigma", lam=1.0)  # Sigma for likelihood function

    # COVID started at different points in different countries, it should be around week 70 give or take a week or two
    covid_start = pm.Data("covid_start", 68)

    # We know from the previous that mu before and during covid should be around 0.8 and 3.0 respectively
    # Sigma is reduced here not to diverge to far from these values
    BoundNormal_0 = pm.Bound(pm.Normal, lower=0)
    weekly_registrations_covid = BoundNormal_0(
        "weekly_registrations_covid", mu=0.8, sigma=0.5
    )
    weekly_registrations_precovid = BoundNormal_0(
        "weekly_registrations_precovid", mu=3, sigma=0.5
    )

    weekly_registrations_base = pm.math.switch(
        covid_start >= model_data.week,
        weekly_registrations_precovid,
        weekly_registrations_covid,
    )

    # Model extra registrations due to shifting interest (like new sets being released)
    # The interest factor is calculated on a weekly basis
    decay_factor = pm.Exponential("decay_factor", lam=1.0)
    interest = [pm.HalfNormal(f"{s}_interest", sigma=2) for s in sets]

    interest_decayed = [interest[0]]
    for i in range(len_observed - 1):
        new_element = interest_decayed[i] * decay_factor

        for ix, wor in enumerate(week_of_release[1:], start=1):
            if i == wor:
                new_element += interest[ix]

        interest_decayed.append(new_element)

    # there were 150k decks registered the first week, that is the initial value
    y0 = tt.zeros(len_observed)
    y0 = tt.set_subtensor(y0[0], 15)

    outputs, _ = theano.scan(
        fn=lambda t, y, ws, intfac: tt.set_subtensor(
            y[t], (ws[t] + weekly_registrations_precovid * intfac[t]) + y[t - 1]
        ),
        sequences=[tt.arange(1, len_observed)],
        outputs_info=y0,
        non_sequences=[weekly_registrations_base, interest_decayed],
        n_steps=len_observed - 1,
    )

    total_registrations = pm.Deterministic("total_registrations", outputs[-1])
    sales_delta = pm.Deterministic(
        "sales_difference",
        np.log2(weekly_registrations_covid / weekly_registrations_precovid),
    )

    # Likelihood
    likelihood = pm.Normal(
        "y", mu=total_registrations, sigma=sigma, observed=model_data.count_scaled
    )




You can find the C code in this temporary file: C:\Users\SEBAST~1\AppData\Local\Temp\theano_compilation_error_u0m1m2a4


Exception: ("Compilation failed (return status=1): C:\\Users\\SEBAST~1\\AppData\\Local\\Temp\\ccp1FzNd.o: In function `run':\r. C:/Users/Sebastian/AppData/Local/Theano/compiledir_Windows-10-10.0.19043-SP0-AMD64_Family_23_Model_113_Stepping_0_AuthenticAMD-3.9.6-64/tmpqub9td00/mod.cpp:99: undefined reference to `__imp__Py_NoneStruct'\r. C:/Users/Sebastian/AppData/Local/Theano/compiledir_Windows-10-10.0.19043-SP0-AMD64_Family_23_Model_113_Stepping_0_AuthenticAMD-3.9.6-64/tmpqub9td00/mod.cpp:124: undefined reference to `__imp_PyExc_ValueError'\r. C:/Users/Sebastian/AppData/Local/Theano/compiledir_Windows-10-10.0.19043-SP0-AMD64_Family_23_Model_113_Stepping_0_AuthenticAMD-3.9.6-64/tmpqub9td00/mod.cpp:130: undefined reference to `__imp_PyExc_RuntimeError'\r. C:/Users/Sebastian/AppData/Local/Theano/compiledir_Windows-10-10.0.19043-SP0-AMD64_Family_23_Model_113_Stepping_0_AuthenticAMD-3.9.6-64/tmpqub9td00/mod.cpp:158: undefined reference to `__imp_PyExc_NotImplementedError'\r. C:/Users/Sebasti

<IPython.core.display.Javascript object>

In [ ]:
with model_6:
    # posterior
    trace_6 = pm.sample(1000, cores=10, chains=4, init="adapt_diag")

In [ ]:
def pickle_model(output_path: str, model, trace):
    """Pickles PyMC3 model and trace"""
    with gzip.GzipFile(output_path, "wb") as file:
        file.write(pickle.dumps({"model": model, "trace": trace}))


pickle_model(
    "./models/model_6.pickle.gz",
    model_6,
    trace_6,
)

In [ ]:
def read_pickle(input_path: str):
    with gzip.GzipFile(input_path, "rb") as file:
        data = file.read()
        unpacked_data = pickle.loads(data)

    return unpacked_data["model"], unpacked_data["trace"]

model_6, trace_6 = read_pickle("./models/model_6.pickle.gz")


In [ ]:
from pymc3_altair import plot_fit_altair

with model_6:
    pm.set_data({"covid_start": 68})

chart = plot_fit_altair(
    model_6,
    trace_6,
    model_data.rename(columns={"week": "Week_nr", "count_scaled": "Total_scaled"}),
    x_range=(1, max(model_data.week)),
)
chart.properties(width="container").save("./altair_output/model_6_covid.json")
chart

In [ ]:
with model_6:
    pm.set_data({"covid_start": 500})

chart_without_covid = plot_fit_altair(
    model_6,
    trace_6,
    model_data.rename(columns={"week": "Week_nr", "count_scaled": "Total_scaled"}),
    x_range=(1, max(model_data.week)),
)
chart_without_covid.properties(width="container").save(
    "./altair_output/model_6_no_covid.json"
)
chart_without_covid

In [ ]:
with model_6:
    sales_baseline_pp = pm.sample_posterior_predictive(
        trace_6,
        var_names=[
            "weekly_registrations_precovid",
            "weekly_registrations_covid",
            "sales_difference",
        ],
        random_seed=873,
    )
sales_baseline_df = pd.DataFrame(sales_baseline_pp).rename(
    columns={
        "weekly_registrations_precovid": "Pre-COVID",
        "weekly_registrations_covid": "COVID",
    }
)
sales_baseline_df

In [ ]:
weekly_registrations = (
    alt.Chart(sales_baseline_df)
    .transform_fold(
        list(sales_baseline_df.columns)[:2],
        as_=["Period", "value"],
    )
    .transform_density(
        density="value", bandwidth=0.3, groupby=["Period"], extent=[0, 3]
    )
    .mark_area(opacity=0.5)
    .encode(
        alt.X("value:Q", title="# Registered Decks Weekly (x10 000)"),
        alt.Y("density:Q", title=""),
        alt.Color("Period:N"),
        tooltip=["Period:N"],
    )
)
weekly_registrations.properties(width="container").save(
    "./altair_output/model_6_weekly_registrations.json"
)
weekly_registrations.properties(width=300, height=200)

In [ ]:
alt.Chart(sales_baseline_df).transform_density(
    density="sales_difference", bandwidth=0.3, extent=[-3, 3]
).mark_area(opacity=0.5).encode(
    alt.X("value:Q", title="Difference in sales (log transformed)"),
    alt.Y("density:Q", title=""),
).properties(
    width=300, height=200
)

In [ ]:
color_range_ = ["#ad2117", "#0f69bd", "#940fbd", "#322d3b", "#5bbab9"]
set_names = [f"{s}_interest" for s in sets]

with model_6:
    set_interest_pp = pm.sample_posterior_predictive(
        trace_6,
        var_names=set_names,
        random_seed=873,
    )
set_interest_df = pd.DataFrame(set_interest_pp).rename(
    columns={a: a.split("_")[0] for a in set_names}
)
set_interest_df

In [ ]:
interest_plot = (
    alt.Chart(set_interest_df)
    .transform_fold(
        list(set_interest_df.columns),
        as_=["Set", "value"],
    )
    .transform_density(density="value", bandwidth=0.3, groupby=["Set"], extent=[0, 6])
    .mark_area(opacity=0.7)
    .encode(
        alt.X("value:Q", title="Popularity"),
        alt.Y("density:Q", title=""),
        alt.Color(
            "Set:N",
            sort=list(set_interest_df.columns),
            scale=alt.Scale(domain=list(set_interest_df.columns), range=color_range_),
        ),
        tooltip=["Set:N"],
    )
)
interest_plot.properties(width="container").save(
    "./altair_output/model_6_set_interest.json"
)

interest_plot.properties(width=300, height=200)